In [26]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

# 1) 데이터 로드 (Kaggle Titanic 기준)
## train_load = pd.read_csv("/kaggle/input/titanic/train.csv")
## test_load  = pd.read_csv("/kaggle/input/titanic/test.csv")
train_load = pd.read_csv("./train.csv")
test_load  = pd.read_csv("./test.csv")


In [39]:
# --- [Feature Engineering Start] ---
def feature_engineering(df):
    df = df.copy()
    
    # 1. Title 추출 (Name에서 호칭만 뽑아내기)
    # 정규표현식: 공백 뒤에 오는 문자열 + 점(.)
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
    
    # 희귀한 호칭들을 묶어서 정리 (Noise Reduction)
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                       'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

    # 각 Title별 평균 나이 계산 (학습 데이터 기준)
    # (실제 값: Master=4.5세, Miss=21세, Mr=32세, Mrs=35세 등)
    title_ages = df.groupby('Title')['Age'].transform('median')
    df['Age'] = df['Age'].fillna(title_ages)
    
    # 2. FamilySize 생성 (SibSp + Parch + 본인 1)
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    # 3. 불필요한 컬럼 제거 (Dimensionality Reduction)
    # Name, Ticket, Cabin은 그대로 OneHotEncoder에 넣으면 
    # 차원이 폭발(Curse of Dimensionality)하므로 반드시 제거해야 합니다.
    df = df.drop(columns=['Name', 'Ticket', 'Cabin'])
    
    return df

train = feature_engineering(train_load)
test  = feature_engineering(test_load)

# --- [Feature Engineering End] ---
# train.head()

In [40]:
# 2) X, y 분리
y = train["Survived"]
X = train.drop(columns=["Survived", "PassengerId"])
X_test = test.drop(columns=["PassengerId"])

# 3) 컬럼 타입 분리 (가장 단순한 자동 규칙)
num_cols = X.select_dtypes(include=["number"]).columns
cat_cols = X.select_dtypes(exclude=["number"]).columns


In [41]:
# 4) 전처리 파이프(수치형/범주형)
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

In [46]:
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop",  # 지정 안 된 컬럼은 버림
)

In [47]:
# from sklearn.ensemble import GradientBoostingClassifier

# # 모델 변경
# model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# clf = Pipeline(steps=[
#     ("preprocess", preprocess),
#     ("model", model),
# ])

# 모델 파이프라인 재설정 (앙상블 말고 이것만 쓰세요)
from sklearn.ensemble import RandomForestClassifier

# 나이가 정확해졌으므로, 트리가 깊을 필요가 없습니다.
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,         # 깊이를 6~7 정도로 제한 (욕심 버리기)
    min_samples_split=10, # 잎사귀 노드에 최소 10개 샘플은 있어야 함 (노이즈 무시)
    random_state=42
)

clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model),
])


In [48]:

# 6) 고정된 CV (StratifiedKFold: 클래스 비율 유지)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(clf, X, y, cv=cv, scoring="accuracy")
print("CV accuracy: %.4f ± %.4f" % (scores.mean(), scores.std()))

CV accuracy: 0.8350 ± 0.0146


In [38]:

# 7) 최종 학습 + 예측 + 제출 파일 생성
clf.fit(X, y)
pred = clf.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": pred.astype(int)
})
submission.to_csv("submission.csv", index=False)
print(submission.head())

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
